In [1]:
import json
import logging
logger = logging.getLogger(__name__)

In [2]:
class QwenDatasets:
    def __init__(self, datapath="../data/qwen-bailian-usagetraces-anon"):
        with open(datapath+"/qwen_traceA_blksz_16.jsonl", "r") as f:
            self.trace_a = f.readlines()
        self.trace_a_size = len(self.trace_a)
        
        with open(datapath+"/qwen_traceB_blksz_16.jsonl", "r") as f:
            self.trace_b = f.readlines()
        self.trace_b_size = len(self.trace_b)
        
        logger.warning(f"loading trace A/B from {datapath}, trace_a_size={self.trace_a_size}, trace_b_size={self.trace_b_size}")
    
    def trace_a(self, idx = None):
        if idx is None:
            idx = 0
            logger.info()
        return self.trace_a
        
    def trace_b(self):
        return self.trace_b

In [3]:
qwen_data = QwenDatasets()

loading trace A/B from ../data/qwen-bailian-usagetraces-anon, trace_a_size=43058, trace_b_size=172800


In [4]:
qwen_data.trace_a[0]

'{"chat_id": 0, "parent_chat_id": -1, "timestamp": 0.0, "input_length": 791, "output_length": 532, "type": "text", "turn": 1, "hash_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]}\n'

In [6]:
import radix_cache as rc